Задача над которой я работаю - Тематическое моделирование. В качестве метрики возьму когерентность. Однако, основным критерием качества будет моя оценка тех топиков, которые выделила модель. Сначала проведу работу с данными, а после использую 2 модели LDA и BERTopic.

In [1]:
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pandas as pd
import re
import pyLDAvis.lda_model
import gensim
import pymorphy3
import requests
from tqdm import tqdm

from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP

morph = pymorphy3.MorphAnalyzer()

Использую набор готовый набор stopwords для русского языка и дополняю его некоторыми словами, которые встречаются почти в каждом объекте 

In [16]:
stopwords_arr = []
with open("russian_stopwords.txt", encoding="UTF-8") as file:
    stopwords_arr = file.read().split("\n")
stopwords_arr += ["цмф", "масло", "маслостанция", "зачаровать", "дон", "как-то"]
len(stopwords_arr)

565

In [17]:
df = pd.read_csv("data_after_preprocess.csv")
df.head()

,text_original,text_after_preprocess
0,«У ннас среди ночi в райооне 55 часов упала по...,У нас среди ночи в районе 55 часов упала полка...
1,Программы повышения квалификации через гильдию...,Программы повышения квалификации через гильдию...
2,"""Мурр... Новиград - город контрастов, да. Но в...","Мурр... Новиград — город контрастов, да. Но в ..."
3,"Типа, вот уже полгода, как мы ждем установки э...","Вот уже полгода, как мы ждём установки этих фи..."
4,"""Теперь всьё так просто! Раньше бумажки летали...","Теперь всё так просто! Раньше бумаги летали, т..."


Функции для работы с данными. После предобрботки с помощью yandexgpt в тексте осталось много символов, которые раздувают размер словаря, от них нужно избавиться. Также нужно убрать слоп-слова, которая не несут в себе информации. И необходимо привести слова к общей форме, чтобы для модели, например, "купил" и "купила" было одним и тем же словом, для этого использую pymorphy3. Также добавляю к тексту биграммы и триграммы.

In [18]:
def count_unique(data): # Подсчёт размера словаря и количесва символов в тексте
    unique_words = set()
    all_len = 0
    for data_item in data:
        if type(data_item) == list:
            data_item = " ".join(data_item)
        unique_words.update(data_item.split(" "))
        all_len += len(data_item)
    print(f"Уникальных слов: {len(unique_words)}")
    print(f"Длина текса: {all_len}")

def remove_bad_symbols(texts): # Оставляю только символы кириллицы в тексте
    for i in range(len(texts)):
        data_item = texts[i].lower()
        new_str = ""
        for letter in data_item:
            if 'а' <= letter <= 'я' or letter == "ё" or letter == " " or letter == "-":
                new_str += letter
        texts[i] = re.sub("  ", " ", new_str)
    return texts
    
def remove_stopwords(texts, stopwords): # Удаляю stopwords
    for i in range(len(texts)):
        data_item = texts[i]
        new_text = []
        for word_arr in data_item:
            for word in word_arr.split(" "):
                if word not in stopwords:
                    new_text.append(word)
        texts[i] = new_text
    return texts

def add_bigram_trigram(texts): # Добавляю биграммы и триграммы
    for i in range(len(texts)):
        texts[i] = texts[i].split(" ")

    bigram = gensim.models.Phrases(texts, min_count=1, threshold=1, delimiter=' ')
    trigram = gensim.models.Phrases(bigram[texts], threshold=1, delimiter=' ')  

    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    new_texts = []
    for i in range(len(texts)):
        new_texts.append(trigram_mod[bigram_mod[texts[i]]])
    return new_texts

def lemmatization(texts): # Привожу слова к нормальной форме
    for i in range(len(texts)):
        for j in range(len(texts[i])):
            new_words = []
            for word in texts[i][j].split(" "):
                new_words.append(morph.parse(word)[0].normal_form)
            texts[i][j] = " ".join(new_words)
    return texts


Размер словаря велик. Посмотрим что получится после обработки данных

In [19]:
data = df["text_after_preprocess"]
count_unique(data)

Уникальных слов: 11811
Длина текса: 253604


In [20]:
data[1]

'Программы повышения квалификации через гильдию ЦМФ хитрые и сложные. У всех, кто хочет развиваться, есть свои предпочтения и направления, а гильдия предлагает свои, не всегда соответствующие потребностям сотрудников.\\n\\nВ «Белых Змеях», например, программы более гибкие, и сотрудники могут сами выбирать то, что им нужно.'

In [21]:
new_data = remove_bad_symbols(data.copy())

new_data = add_bigram_trigram(new_data)

new_data = lemmatization(new_data)

new_data = remove_stopwords(new_data, stopwords_arr)

print(new_data[1])

['программа', 'повышение', 'квалификация', 'гильдия', 'хитрый', 'сложный', 'развиваться', 'предпочтение', 'направление', 'гильдия', 'предлагать', 'соответствующий', 'потребность', 'сотрудниковва', 'змея', 'например', 'программа', 'гибкий', 'сотрудник', 'выбирать']


In [22]:
count_unique(new_data)

Уникальных слов: 4412
Длина текса: 130530


Размер словаря сократился почти в 3 раза\
Обучим LDA и подберём лучшее количество тематик по когерентности

In [23]:
def find_best_lda(texts):
    best_coherence_lda = 0
    best_lda = None
    best_num_topics = -1
    
    id2word = gensim.corpora.Dictionary(texts)
    corpus = [id2word.doc2bow(text) for text in texts]
    
    for i in tqdm(range(5, 15)):
        print(0)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                    id2word=id2word,
                                                    num_topics=i)
        
        coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        
        if coherence_lda > best_coherence_lda:
            best_coherence_lda = coherence_lda
            best_lda = lda_model
            best_num_topics = i
    return best_lda, best_num_topics, best_coherence_lda

In [ ]:
lda_model, best_topics_size, best_coherence = find_best_lda(new_data)

  0%|                                                                                                                                                       | 0/10 [00:00<?, ?it/s]

In [91]:
print('Coherence Score: ', best_coherence)

Coherence Score:  0.3879314304293448


In [92]:
print(f"Лучшая модель выделила {best_topics_size} тематик")

Лучшая модель выделила 14 тематик


Визуализируем тематики и посмотрим на их информативность

In [93]:
id2word = gensim.corpora.Dictionary(new_data)

corpus = [id2word.doc2bow(text) for text in new_data]

In [94]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10     0.010297  0.050120       1        1  9.661878
7      0.023150  0.060192       2        1  9.460848
6     -0.121718  0.004248       3        1  9.054032
0     -0.032539 -0.065223       4        1  8.377668
5      0.009867  0.106679       5        1  8.086375
11     0.030653 -0.026739       6        1  7.656870
4      0.036083  0.027403       7        1  7.321815
2     -0.122863 -0.025689       8        1  7.281671
13    -0.024590  0.048661       9        1  6.473875
9      0.046043  0.047456      10        1  5.890427
3      0.060562 -0.083240      11        1  5.633656
8      0.041597 -0.044202      12        1  5.224104
12     0.036857 -0.044401      13        1  5.050649
1      0.006602 -0.055266      14        1  4.826132, topic_info=            Term       Freq      Total Category  logprob  loglift
477         тема  20.000000  20.000000  Default  30.0000  30.0000
474      найтись  17.000000  17.000000  Default  29.0000  29.0000
475        поиск  17.000000  17.000000  Default  28.0000  28.0000
473     интернет  17.000000  17.000000  Default  27.0000  27.0000
476         сайт  18.000000  18.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
110      условие   2.457446  42.636204  Topic14  -5.6982   0.1775
99      зарплата   2.481594  66.199363  Topic14  -5.6884  -0.2526
24      например   2.263863  74.145642  Topic14  -5.7803  -0.4578
586    приходить   2.173949  28.148348  Topic14  -5.8208   0.4702
70   приходиться   2.191596  50.914428  Topic14  -5.8127  -0.1144

[1099 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3976      7  0.667798  аварийность
3977      7  0.667810       авария
1603     11  0.322984           аз
1194      1  0.480561  аккумулятор
1194      2  0.160187  аккумулятор
...     ...       ...          ...
439       3  0.232286   эльфийский
439       4  0.232286   эльфийский
439       7  0.116143   эльфийский
439       9  0.232286   эльфийский
2990     14  0.690310           яд

[2976 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 8, 7, 1, 6, 12, 5, 3, 14, 10, 4, 9, 13, 2])

Качество тематик мне не нравится. Среди них есть информативные, но в большинстве своём они похожи по наполнению.\
Попробуем найти что-то поинтереснее с помощью BERTopic

In [108]:
def count_coherence(docs, topic_model):
    vectorizer = topic_model.vectorizer_model
    analyzer = vectorizer.build_analyzer()

    words = vectorizer.get_feature_names_out()
    tokens = [analyzer(doc) for doc in docs]
    dictionary = gensim.corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
                   for topic in range(len(set(topics))-1)]

    coherence_model = CoherenceModel(topics=topic_words, 
                                     texts=tokens, 
                                     corpus=corpus,
                                     dictionary=dictionary, 
                                     coherence='c_v')
    return coherence_model.get_coherence()

In [97]:
df = pd.read_csv("data_after_preprocess.csv")
df.head()

,text_original,text_after_preprocess
0,«У ннас среди ночi в райооне 55 часов упала по...,У нас среди ночи в районе 55 часов упала полка...
1,Программы повышения квалификации через гильдию...,Программы повышения квалификации через гильдию...
2,"""Мурр... Новиград - город контрастов, да. Но в...","Мурр... Новиград — город контрастов, да. Но в ..."
3,"Типа, вот уже полгода, как мы ждем установки э...","Вот уже полгода, как мы ждём установки этих фи..."
4,"""Теперь всьё так просто! Раньше бумажки летали...","Теперь всё так просто! Раньше бумаги летали, т..."


Проделываю работу с данными

In [99]:
data = list(df["text_after_preprocess"])

new_data = remove_bad_symbols(data.copy())

new_data = add_bigram_trigram(new_data)

new_data = lemmatization(new_data)

new_data = remove_stopwords(new_data, stopwords_arr)

for i in range(len(new_data)):
    new_data[i] = " ".join(new_data[i])
    
print(new_data[1])

программа повышение квалификация гильдия хитрый сложный развиваться предпочтение направление гильдия предлагать соответствующий потребность сотрудниковва змея например программа гибкий сотрудник выбирать


In [103]:
umap = UMAP(n_neighbors=15,
                n_components=5,
                min_dist=0.0,
                metric='cosine',
                low_memory=False,
                random_state=3) 

model = BERTopic(
    umap_model=umap,
    language='russian', calculate_probabilities=True,
    verbose=False
)

topics, probs = model.fit_transform(new_data)
freq = model.get_topic_info()
freq

,Topic,Count,Name,Representation,Representative_Docs
0,-1,294,-1_то_какой_например_что,"[то, какой, например, что, премия, сотрудник, ...",[отлично бандит бояться подходить змс руническ...
1,0,80,0_зарплата_тысяча_условие_премия,"[зарплата, тысяча, условие, премия, конкуренци...",[гильдия огненный нильфгаард молодой специалис...
2,1,63,1_то_собеседование_смена_управлять,"[то, собеседование, смена, управлять, руководи...",[влиять высказывание клиент например ой ладный...
3,2,54,2_дракон_змеиный_красный_огненный,"[дракон, змеиный, красный, огненный, зачарован...",[гвиндефа воздух вибрировать энергия магия поя...
4,3,49,3_алхимик_образование_подмастерье_обучение,"[алхимик, образование, подмастерье, обучение, ...",[идея внутренний резерв сотрудник учёт переезд...
5,4,48,4_график_ночной_трасса_добираться,"[график, ночной, трасса, добираться, смена, ез...",[основное слушать спрашивать добираться пробле...
6,5,38,5_зарплата_условие_огненный_платить,"[зарплата, условие, огненный, платить, дракон,...",[ценник масляный станция обновить неравномерно...
7,6,38,6_снежный_королева_сто_мойка,"[снежный, королева, сто, мойка, снег, открыват...",[план удовольствие документ отчёт снежный коро...
8,7,37,7_кофе_касса_туалет_зал,"[кофе, касса, туалет, зал, готовить, стоить, е...",[подходить доказывать забыть картофель фри заб...
9,8,35,8_праздник_рождение_отмечать_коллега,"[праздник, рождение, отмечать, коллега, по, пр...",[амарильо общий получиться праздник рабочий об...


BERTopic выделил много информативных тематик. Замерим когерентность.

In [109]:
count_coherence(new_data, model)

0.4239301821550942

Рассмотрим другой подход к работе с данными и замерим качество модели

In [111]:
data = list(df["text_after_preprocess"])

new_data = remove_bad_symbols(data.copy())

for i in range(len(new_data)):
    new_data[i] = new_data[i].split(" ")

new_data = lemmatization(new_data)

for i in range(len(new_data)):
    new_data[i] = " ".join(new_data[i])
    
print(new_data[1])

программа повышение квалификация через гильдия цмф хитрый и сложный у весь кто хотеть развиваться есть свой предпочтение и направление а гильдия предлагать свой не всегда соответствующий потребность сотрудниковва белый змея например программа более гибкий и сотрудник мочь сам выбирать то что они нужно


In [113]:
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stopwords_arr)

umap = UMAP(n_neighbors=15,
                n_components=5,
                min_dist=0.0,
                metric='cosine',
                low_memory=False,
                random_state=3) 

model = BERTopic(
    vectorizer_model=vectorizer_model,
    umap_model=umap,
    language='russian', calculate_probabilities=True,
    verbose=False
)

topics, probs = model.fit_transform(new_data)
freq = model.get_topic_info()
freq

,Topic,Count,Name,Representation,Representative_Docs
0,-1,55,-1_палантин_канцелярия_чистый_обед,"[палантин, канцелярия, чистый, обед, скриптори...",[трупалантир всё частый давать сбой связь с фи...
1,0,794,0_зарплата_например_смена_компания,"[зарплата, например, смена, компания, вроде, с...",[два год назад случиться кризис и я понять что...
2,1,29,1_праздник_рождение_отмечать_рождение коллега,"[праздник, рождение, отмечать, рождение коллег...",[так сказать ранний когда мы только начинать з...
3,2,25,2_планшет_аккумулятор_приходиться_аккумулятор ...,"[планшет, аккумулятор, приходиться, аккумулято...",[мур-мур этот планшет от цмф просто чудо аккум...
4,3,22,3_тема найтись поиск_информация тема найтись_н...,"[тема найтись поиск, информация тема найтись, ...",[в интернет есть много сайт с информация на эт...
5,4,20,4_мёд_топлёный_топлёный мёд_продажа,"[мёд, топлёный, топлёный мёд, продажа, томлёны...",[ааа марибора вспоминать то время просто ух пр...
6,5,14,5_зеркало_слетать_словно_мур,"[зеркало, слетать, словно, мур, прицел, волшеб...",[мур-р этот зеркало интересно как они это дела...


In [114]:
count_coherence(new_data, model)

0.5783559870939489

Работу над удалением стоп-слов и созданием ngram я предоставил CountVectorizer. Когерентность выросла, но качество тематик у прошлого варианта мне нравятся больше. По этой причине используем её.

In [115]:
data = list(df["text_after_preprocess"])

new_data = remove_bad_symbols(data.copy())

new_data = add_bigram_trigram(new_data)

new_data = lemmatization(new_data)

new_data = remove_stopwords(new_data, stopwords_arr)

for i in range(len(new_data)):
    new_data[i] = " ".join(new_data[i])

umap = UMAP(n_neighbors=15,
                n_components=5,
                min_dist=0.0,
                metric='cosine',
                low_memory=False,
                random_state=3) 

model = BERTopic(
    umap_model=umap,
    language='russian', calculate_probabilities=True,
    verbose=False
)

topics, probs = model.fit_transform(new_data)
freq = model.get_topic_info()

In [227]:
best_model = model
freq.head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,294,-1_то_какой_например_что,"[то, какой, например, что, премия, сотрудник, ...",[отлично бандит бояться подходить змс руническ...
1,0,80,0_зарплата_тысяча_условие_премия,"[зарплата, тысяча, условие, премия, конкуренци...",[гильдия огненный нильфгаард молодой специалис...
2,1,63,1_то_собеседование_смена_управлять,"[то, собеседование, смена, управлять, руководи...",[влиять высказывание клиент например ой ладный...
3,2,54,2_дракон_змеиный_красный_огненный,"[дракон, змеиный, красный, огненный, зачарован...",[гвиндефа воздух вибрировать энергия магия поя...
4,3,49,3_алхимик_образование_подмастерье_обучение,"[алхимик, образование, подмастерье, обучение, ...",[идея внутренний резерв сотрудник учёт переезд...


In [228]:
freq.loc[15, "Representative_Docs"]

['процесс документооборот планшет вещь кардинальный ранний вручную бумажка толкать электронный быстрый ошибка меньшеоднако карбонск привыкнуть равно предпочитать бумажкиа змеиный дыхание планшет быстрый привыкнуть',
 'исправить текстцк например планшет огонь аккумулятор держать выручать',
 'вроде неплохой планшет мобильный вроде удобный аккумулятор держать недолго хотеться снежный королева планшет целый']

С помощью yandexgpt обработаю ключевые слова для каждого топика и создам название тематик

In [197]:
prompt = {
    "modelUri": "gpt://мой токен/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.1,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "user",
            "text": ""
        }
    ]
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key мой токен"
}

In [198]:
topic_to_name = {}

In [199]:
for i in range(len(freq)):
    info = freq.loc[i, "Representation"]
    new_text = ", ".join(info)
    
    prompt_text = "Составь короткое названия темы, в которую входят эти слова и ничего больше не пиши: "
    
    prompt["messages"][0]["text"] = prompt_text + new_text
    response = requests.post(url, headers=headers, json=prompt)
    result = response.text

    start_index = result.find('"text":') + len('"text":')
    end_index = result.find('"},"status')
    new_text = result[start_index+1:end_index]
    
    topic_to_name[i - 1] = new_text

Изменю "-1" топик, так как в него вошли объекты, для которых модель не определила тематику

In [201]:
topic_to_name[-1] = "Тематика не определена"

In [202]:
topic_to_name

{-1: 'Тематика не определена',
 0: 'Финансы',
 1: 'Управление компанией',
 2: 'Мифические существа',
 3: 'Образование и химия',
 4: 'Транспорт и перемещение',
 5: 'Финансы',
 6: 'Зима',
 7: 'Общепит',
 8: 'Рождение и праздники',
 9: 'Орфография и ошибки',
 10: 'Нефтепродукты',
 11: 'Продажа мёда',
 12: 'Поиск информации в интернете',
 13: 'Объекты и сравнения',
 14: 'Техника и обслуживание',
 15: 'Воздух и вентиляция',
 16: 'Грызуны',
 17: 'Медицина',
 18: 'Канцелярия и документы',
 19: 'Одежда'}

Cопоставим каждому объекту из наших данных номер тематики и сохраним информацию в виде csv файла.

In [206]:
df = pd.read_csv("data_after_preprocess.csv")
df["topic_index"] = [None] * len(df)
df["topic_name"] = [None] * len(df)
df.head()

,text_original,text_after_preprocess,topic_index,topic_name
0,«У ннас среди ночi в райооне 55 часов упала по...,У нас среди ночи в районе 55 часов упала полка...,None,None
1,Программы повышения квалификации через гильдию...,Программы повышения квалификации через гильдию...,None,None
2,"""Мурр... Новиград - город контрастов, да. Но в...","Мурр... Новиград — город контрастов, да. Но в ...",None,None
3,"Типа, вот уже полгода, как мы ждем установки э...","Вот уже полгода, как мы ждём установки этих фи...",None,None
4,"""Теперь всьё так просто! Раньше бумажки летали...","Теперь всё так просто! Раньше бумаги летали, т...",None,None


In [216]:
for i in range(len(new_data)):
    topic, _ = model.transform(new_data[i])
    topic = topic[0]
    df.at[i, "topic_index"] = topic
    df.at[i, "topic_name"] = topic_to_name[topic]

In [222]:
df.head()

,text_original,text_after_preprocess,topic_index,topic_name
0,«У ннас среди ночi в райооне 55 часов упала по...,У нас среди ночи в районе 55 часов упала полка...,13,Объекты и сравнения
1,Программы повышения квалификации через гильдию...,Программы повышения квалификации через гильдию...,3,Образование и химия
2,"""Мурр... Новиград - город контрастов, да. Но в...","Мурр... Новиград — город контрастов, да. Но в ...",-1,Тематика не определена
3,"Типа, вот уже полгода, как мы ждем установки э...","Вот уже полгода, как мы ждём установки этих фи...",10,Нефтепродукты
4,"""Теперь всьё так просто! Раньше бумажки летали...","Теперь всё так просто! Раньше бумаги летали, т...",14,Техника и обслуживание


In [223]:
df.to_csv("answer.csv", index=False)

In [224]:
df_test = pd.read_csv("answer.csv")
df_test.head()

,text_original,text_after_preprocess,topic_index,topic_name
0,«У ннас среди ночi в райооне 55 часов упала по...,У нас среди ночи в районе 55 часов упала полка...,13,Объекты и сравнения
1,Программы повышения квалификации через гильдию...,Программы повышения квалификации через гильдию...,3,Образование и химия
2,"""Мурр... Новиград - город контрастов, да. Но в...","Мурр... Новиград — город контрастов, да. Но в ...",-1,Тематика не определена
3,"Типа, вот уже полгода, как мы ждем установки э...","Вот уже полгода, как мы ждём установки этих фи...",10,Нефтепродукты
4,"""Теперь всьё так просто! Раньше бумажки летали...","Теперь всё так просто! Раньше бумаги летали, т...",14,Техника и обслуживание


Моё решение хорошо тем, что даёт много информации для дальнейшего анализа. Можно достать из "csv" файла какую-либо тематику и подробно разобраться в том какая проблемма есть у компании или какие преимущества люди видят в ней. Пример с одной из тематик ("Техника и обслуживание"):

In [240]:
list(df_test[df_test["topic_index"] == 14]["text_after_preprocess"])[:6]

['Теперь всё так просто! Раньше бумаги летали, терялись, и их приходилось искать по всему Зелёному городу. А теперь всё в одном месте, на планшете. Вся информация под рукой, ничего не пропадает. Работать стало гораздо удобнее!',
 'Эти планшеты всё время дают сбои. Всё время что-то глючит. Вместо того чтобы быстро оформить, приходится стоять и ждать, пока он сам не решит, что всё в порядке. И информация, кажется, не хочет туда заходить. Быстрее было раньше, когда всё делали руками.',
 'Конкретно, вот уже полгода мечтаю о том, чтобы у нас на кассе появился хотя бы один палантин, который бы заряжался как раз на целый день, а то эти аккумуляторчики, которые сейчас есть, конкретно раздражают.',
 'С такими темпами, как мы идём, скоро и планшеты стационарные заменят... вот только «Огненные Скорпионы» уж больно дерзко заявляют, что им удалось создать что-то подобное, но я... я всё-таки верю в ЦМФ!',
 'Палантир — штука неудобная. Слишком много лишних движений: взять, ввести, проверить, ещё раз 

Можно заметить какие проблемы у пользователей возникают с техникой. Также для анализа удобно, что есть признак "text_after_preprocess", так как читать исходные данные тяжело

Что я бы ещё сделал, но не успел:
Нужна работа с результатами, так как некоторые тематики схожи между собой. Нужна более детальная настройка BERTopic и больший анализ результатов.\
Для предобработки текста попробывал бы использовать другие gpt и сравнить качество результатов